In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/soccer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///../input/soccer/database.sqlite")
engine.table_names()

### **Number of Matches played in the 11 leagues**

In [6]:
pd.read_sql_query('''SELECT name, COUNT(country_id) AS matches
                     FROM league
                     LEFT JOIN Match
                     USING(country_id)
                     GROUP BY name
                     LIMIT 5;''', engine)

Suppose we want to compare the number of 
* home team wins, 
* away team wins,
* ties

In [7]:
pd.read_sql_query('''SELECT date, id, home_team_goal, away_team_goal
                     FROM Match 
                     WHERE season = '2013/2014'
                     AND home_team_goal > away_team_goal
                     LIMIT 4;  ''', engine)

This is not efficient so we will use
# CASE statements

* `WHEN`, `THEN`, `ELSE`, `END`

In [8]:
pd.read_sql_query('''SELECT id, home_team_goal, away_team_goal,
                     CASE WHEN home_team_goal > away_team_goal THEN 'Home Team Win'
                          WHEN home_team_goal < away_team_goal THEN 'Away Team Win'
                          ELSE 'Tie' END AS outcome
                     FROM Match 
                     WHERE season = '2013/2014'
                     LIMIT 4;  ''', engine)

### **Basic CASE statements**
What is your favorite team?

The *European Soccer Database* contains data about 12,800 matches from 11 countries played between 2011-2015! Throughout this course, you will be shown filtered versions of the tables in this database in order to better explore their contents.

In this exercise, you will identify matches played between *FC Schalke 04* and *FC Bayern Munich*. There are 2 teams identified in each match in the `hometeam_id` and `awayteam_id` columns, available to you in the `Match` table. ID can join to the `team_api_id` column in the `Team` table, but you cannot perform a join on both at the same time.

However, you can perform this operation using a `CASE` statement once you've identified the `team_api_id` associated with each team!

* Select the team's long name and API id from the `Team` table.
* Filter the query for *FC Schalke 04* and *FC Bayern Munich* using `IN`, giving you the `team_api_id`s needed for the next step.

In [9]:
pd.read_sql_query('''SELECT team_long_name,
                            team_api_id
                     FROM Team
                     WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');''', engine)

* Create a `CASE` statement that identifies whether a match in Germany included `FC Bayern Munich`, `FC Schalke 04`, or neither as the home team.
* Group the query by the `CASE` statement alias, `home_team`.

In [10]:
pd.read_sql_query('''SELECT CASE WHEN home_team_api_id = 10189 THEN 'FC Schalke 04'
                                 WHEN away_team_api_id = 9823 THEN 'FC Bayern Munich'
                                 ELSE 'Other' END AS home_team,
                            COUNT(id) AS total_matches
                      FROM Match
                      GROUP BY home_team; ''', engine)

### **CASE statements comparing column values**

Barcelona is considered one of the strongest teams in Spain's soccer league.

In this exercise, you will be creating a list of matches in the 2011/2012 season where Barcelona was the home team. You will do this using a `CASE` statement that compares the values of two columns to create a new group -- wins, losses, and ties.

In 3 steps, you will build a query that identifies a match's winner, identifies the identity of the opponent, and finally filters for Barcelona as the home team. Completing a query in this order will allow you to watch your results take shape with each new piece of information.

The `Match` table currently contains  matches from the 2011/2012 season, and has two key columns, `home_team_api_id` and `away_team_api_id`, that can be joined with the `Team` table. However, you can only join `Team` to one column at a time.

* Select the `date` of the match and create a `CASE` statement to identify matches as home wins, home losses, or ties.

In [11]:
pd.read_sql_query('''SELECT date,
                            CASE WHEN home_team_goal > away_team_goal THEN 'Home win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Home loss :(' 
                                 ELSE 'Tie' END AS outcome
                     FROM Match
                     WHERE season = '2011/2012'
                     LIMIT 5;''', engine)

* Left join the `Team` table `team_api_id` column to the `Match` table `away_team_api_id`. This allows us to retrieve the *away* team's identity.
* Select `team_long_name` from `Team` as `opponent` and complete the `CASE` statement from Step 1.

In [12]:
pd.read_sql_query('''SELECT date,
	                        team_long_name AS opponent, 
	                        CASE WHEN home_team_goal > away_team_goal THEN 'Home win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Home loss :('
                                 ELSE 'Tie' END AS outcome
                      FROM Match 
-- Left join Team onto Match
                      LEFT JOIN Team 
                      ON away_team_api_id = team_api_id
                      WHERE season = '2011/2012' LIMIT 4;''', engine)

* Modify the same `CASE` statement as the previous steps.
* Filter for matches where the home team is FC Barcelona (id = `8634`).

In [13]:
pd.read_sql_query('''SELECT date,
	                        team_long_name AS opponent, 
	                        CASE WHEN home_team_goal > away_team_goal THEN 'Barcelona win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Barcelona loss :('
                                 ELSE 'Tie' END AS outcome
                      FROM Match
                      LEFT JOIN Team
                      ON away_team_api_id = team_api_id
                      WHERE season = '2011/2012'
                      AND home_team_api_id = 8634;''', engine)

### **CASE statements comparing two column values part 2**

Similar to the previous exercise, you will construct a query to determine the outcome of Barcelona's matches where they played as the *away team*. You will learn how to combine these two queries soon below.

Did their performance differ from the matches where they were the home team?

* Complete the `CASE` statement to identify Barcelona's away team games (`id = 8634`) as wins, losses, or ties.
* Left join the `Team` table `team_api_id` column on the `Match` table `home_team_id` column. This retrieves the identity of the <ins>*home team*</ins> opponent.
* Filter the query to only include matches where Barcelona was the *away* team.

In [14]:
pd.read_sql_query('''SELECT date, team_long_name AS opponent,
                            CASE WHEN home_team_goal < away_team_goal THEN 'Barcelona win!'
                                 WHEN home_team_goal > away_team_goal THEN 'Barcelona loss :(' 
                                 ELSE 'Tie' END AS outcome
                     FROM Match m
                     LEFT JOIN Team t
                     ON m.home_team_api_id = t.team_api_id
                     WHERE season = '2011/2012'
                     AND m.away_team_api_id = 8634;''', engine)

## **Reviewing `CASE` `WHEN`**

In [15]:
pd.read_sql_query('''SELECT date, season,
                            CASE WHEN home_team_goal > away_team_goal THEN 'Home team win!'
                                 WHEN home_team_goal < away_team_goal THEN 'Away team win'
                                 ELSE 'Tie' END AS outcome
                            FROM Match 
                            LIMIT 5''', engine)

# **Multiple Logical Conditions**
## **CASE WHEN AND then some ...**

* Use `AND` inside the `WHEN` clause

In [16]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 ELSE 'Loss or Tie :(' END AS outcome
                     FROM Match
                     WHERE home_team_api_id = 8455 OR away_team_api_id = 8455
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

ถ้าเอา `WHERE home_team_api_id = 8455` OR `away_team_api_id = 8455`ออก
ก็จะกลายเป็นว่า ผลลัพธ์คู่อื่นๆเป็น แพ้หรือเสมอ กันทั้งหด

In [17]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 ELSE 'Loss or Tie :(' END AS outcome
                     FROM Match
                     WHERE season = '2011/2012'
                     LIMIT 10''', engine)

## **FILTER query by CASE**

* ถ้าแค่เอา ELSE ออกไป จะมีค่า NULL โผล่มาเยอะ



In [18]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END AS outcome
                     FROM Match
                     WHERE home_team_api_id = 8455 OR away_team_api_id = 8455
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

* Include entire CASE without alias in WHERE
* ให้สังเกตว่า ไม่ต้องใส่ `WHERE home_team_api_id = 8455 OR away_team_api_id = 8455`แล้ว

In [19]:
pd.read_sql_query('''SELECT date,season, home_team_api_id, away_team_api_id,
                            CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END AS outcome
                     FROM Match
                     WHERE CASE WHEN home_team_goal > away_team_goal AND home_team_api_id = 8455 THEN 'Chelsea home win!'
                                 WHEN home_team_goal < away_team_goal AND away_team_api_id = 8455 THEN 'Chelsea away win'
                                 END IS NOT NULL
                     AND season = '2011/2012'
                     LIMIT 10''', engine)

## **ก่อนอื่น ลองทำตาราง teams_spain และ matches_spain ตามด้านล่างนี้ก่อน จะได้ไม่ต้องทำจากข้อมูลทั้งหมดเหมือนด้านบน และจะได้ได้ผลลัพธ์ใกล้เคียงกับที่แสดงให้เห็นในแบบฝึกหัดด้วย**

In [20]:
# teams_spain 
# จริงๆต้องเชื่อมด้วยตาราง Match ด้วย Country_id และเชื่อมต่อด้วยตาราง Country ด้วย id และกรองด้วยคำว่า WHERE country = 'Spain'
# แต่มันออกมาต่างกันนิดหน่อย เลยใช้วิธีนี้ให้เหมือนมากกว่า
pd.read_sql('''SELECT id, Team_api_id, Team_long_name, Team_short_name
               FROM Team
               WHERE id BETWEEN 43035 AND 47615
               AND id NOT IN (43041, 43048, 43800, 43804, 44557) 
               ORDER BY id LIMIT 50''', engine)

In [21]:
# matches_spain
pd.read_sql_query('''SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                     FROM Match
                     WHERE id BETWEEN 22658 AND 24177''', engine)

### **In CASE of rivalry**

Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name *El Clásico* (The Classic). In this exercise, you will query a list of matches played between these two rivals.

You will notice in Step 2 that when you have multiple logical conditions in a `CASE` statement, you may quickly end up with a large number of `WHEN` clauses to logically test every outcome you are interested in. It's important to make sure you don't accidentally exclude key information in your `ELSE` clause.

In this exercise, you will retrieve information about matches played between **Barcelona** (id = `8634`) and **Real Madrid** (id = `8633`). Note that the query you are provided with already identifies the *Clásico* matches using a filter in the `WHERE` clause.

* Complete the first `CASE` statement, identifying Barcelona or Real Madrid as the home team using the `hometeam_id` column.
* Complete the second `CASE` statement in the same way, using `awayteam_id`.

In [22]:
pd.read_sql_query('''SELECT date, 
	-- Identify the home team as Barcelona or Real Madrid
                            CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
                                 ELSE 'Real Madrid CF' END AS home,
    -- Identify the away team as Barcelona or Real Madrid
                            CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
                                 ELSE 'Real Madrid CF' END AS away
    -- ตรงนี้ แทนที่ matche_spain ในแบบฝึกหัดด้วย subqueries
                     FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                           FROM Match
                           WHERE id BETWEEN 22658 AND 24177) AS matches_spain
                     WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
                     AND (awayteam_id = 8633 OR hometeam_id = 8633);''', engine)

* Construct the final `CASE` statement identifying who won each match. Note there are 3 possible outcomes, but 5 conditions that you need to identify.
* Fill in the logical operators to identify Barcelona or Real Madrid as the winner.

In [23]:
pd.read_sql_query('''SELECT date, 
                            CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' ELSE 'Real Madrid CF' END as home,
                            CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' ELSE 'Real Madrid CF' END as away,
	-- Identify all possible match outcomes
                            CASE WHEN home_goal  > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
                            WHEN home_goal  > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
                            WHEN home_goal  < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
                            WHEN home_goal  < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
                            ELSE 'Tie!' END AS outcome
                     FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                           FROM Match
                           WHERE id BETWEEN 22658 AND 24177) AS matches_spain
                     WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
                     AND (awayteam_id = 8633 OR hometeam_id = 8633);''', engine)

แบบฝึกหัดถัดไปใช้ข้อมูลของอิตาลี


In [24]:
# teams_italy
pd.read_sql('''SELECT id, Team_api_id, Team_long_name, Team_short_name
               FROM Team
               WHERE id BETWEEN 20513 AND 25048
               AND id NOT IN (20520, 21280, 22044) 
               ORDER BY id LIMIT 50''', engine)

In [25]:
# matches_italy
pd.read_sql_query('''SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                     FROM Match
                     WHERE id BETWEEN 11397 AND 12893''', engine)

## **Filtering your `CASE` statement**

Let's generate a list of matches <ins>won</ins> by Italy's *Bologna* team! There are quite a few additional teams in the two tables, so a key part of generating a usable query will be using your `CASE` statement as a filter in the `WHERE` clause.

`CASE` statements allow you to categorize data that you're interested in -- and exclude data you're not interested in. In order to do this, you can use a `CASE` statement as a filter in the `WHERE` statement to remove output you don't want to see.

Here is how you might set that up:

```
SELECT *
FROM table
WHERE 
    CASE WHEN a > 5 THEN 'Keep'
         WHEN a <= 5 THEN 'Exclude' END = 'Keep';
```         
         
         
In essence, you can use the `CASE` statement as a filtering column like any other column in your database. The only difference is that <ins>you don't alias the statement in `WHERE`</ins>.

* Identify `Bologna`'s team ID listed in the `teams_italy` table by selecting the `team_long_name` and `team_api_id`.

In [26]:
pd.read_sql_query('''SELECT team_long_name, team_api_id
                     FROM (SELECT id, Team_api_id, Team_long_name, Team_short_name
                           FROM Team
                           WHERE id BETWEEN 20513 AND 25048
                           AND id NOT IN (20520, 21280, 22044) 
                           ORDER BY id) AS teams_italy
                     WHERE team_long_name = 'Bologna';''', engine)

* Select the `season` and `date` that a match was played.
* Complete the `CASE` statement so that only Bologna's home and away wins are identified.

In [27]:
pd.read_sql_query('''SELECT season, date, CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
                                               WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
                                               END AS outcome
                      FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                            FROM Match
                            WHERE id BETWEEN 11397 AND 12893) AS matches_italy;''', engine)

* Select the `home_goal` and `away_goal` for each match.
* Use the `CASE` statement in the `WHERE` clause to filter all `NULL` values generated by the statement in the previous step.

In [28]:
pd.read_sql_query('''
SELECT season, date, home_goal,	away_goal,
       CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Home Win'
		    WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Away Win' 
            END AS outcome

FROM (SELECT id,	country_id,	season,	stage, date, home_team_api_id AS hometeam_id, away_team_api_id AS awayteam_id, home_team_goal AS home_goal, away_team_goal AS away_goal
                            FROM Match
                            WHERE id BETWEEN 11397 AND 12893) AS matches_italy
WHERE 
-- Exclude games not won by Bologna
	CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
		 WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
		 END IS NOT NULL;''', engine)

## **Counting CASES**

* Count the number of home and away games that Liverpool won in each season
* `CASE` สามารถนำไปใส่ใน aggregate functions ได้

## **CASE WHEN with COUNT**

* จุดที่แตกต่างคือสิ่งที่เขียนตามหลัง `THEN` แทนที่จะเขียนเป็นตัวหนังสือ ให้ใช้คอลัมน์ที่ไม่มีค่าซ้ำ เช่น `id`
    * จริงๆแล้วจะเป็นค่าอะไรก็ได้
* พอ `CASE` นี้ไปอยู่ใน `COUNT` ก็เลยเป็นการนับค่าตามเงื่อนไขที่เกิดขึ้นนั่นเอง

In [29]:
pd.read_sql_query('''
SELECT season,
       COUNT(CASE WHEN home_team_api_id = 8650 AND home_team_goal > away_team_goal THEN id END) AS home_wins,
       COUNT(CASE WHEN away_team_api_id = 8650 AND home_team_goal < away_team_goal THEN id END) AS away_wins
FROM Match
GROUP BY season;''', engine)

# **CASE WHEN with SUM**

* สมมติสนใจจำนวนประตูที่ลิเวอร์พูลทำแต้มได้ในแต่ละฤดูกาล

In [30]:
pd.read_sql_query('''
SELECT season,
       SUM(CASE WHEN home_team_api_id = 8650 THEN home_team_goal END) AS home_goals,
       SUM(CASE WHEN away_team_api_id = 8650 THEN away_team_goal END) AS away_goals
FROM Match
GROUP BY season;''', engine)

# **CASE with AVG (1)**

In [31]:
pd.read_sql_query('''
SELECT season,
       ROUND(AVG(CASE WHEN home_team_api_id = 8650 THEN home_team_goal END),2) AS home_goals,
       ROUND(AVG(CASE WHEN away_team_api_id = 8650 THEN away_team_goal END),2) AS away_goals
FROM Match
GROUP BY season;''', engine)

# **Percentages with CASE and AVG**

* หาร้อยละที่เชลซีชนะ

In [32]:
pd.read_sql_query('''
SELECT season, team_long_name,
       ROUND(AVG(CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 1 
                      WHEN home_team_api_id = 8455 AND home_team_goal < away_team_goal THEN 0
                      END), 2) AS pct_homewins,
       ROUND(AVG(CASE WHEN away_team_api_id = 8455 AND home_team_goal < away_team_goal THEN 1 
                      WHEN away_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 0
                      END), 2) AS pct_awaywins
FROM Match
INNER JOIN Team ON (Team.team_api_id = Match.home_team_api_id) OR (Team.team_api_id = Match.away_team_api_id)
WHERE Team.team_api_id = 8455
GROUP BY season;''', engine)

### **COUNT using CASE WHEN**

Do the number of soccer matches played in a given European country differ across seasons? 

You will examine the number of matches played in 3 seasons within each country listed in the database. This is much easier to explore with each season's matches in separate columns. Using the `Country` and unfiltered `Match` table, you will count the number of matches played in each country during the 2012/2013, 2013/2014, and 2014/2015 match seasons.

* Create a `CASE` statement that identifies the `id` of matches played in the 2012/2013 season. Specify that you want `ELSE` values to be `NULL`.
* Wrap the `CASE` statement in a `COUNT` function and group the query by the `country` alias.

In [33]:
pd.read_sql_query('''
SELECT c.name AS country,
    -- Count games from the 2012/2013 season
       COUNT(CASE WHEN m.season = '2012/2013' THEN 'whatever' ELSE NULL END) AS matches_2012_2013
FROM Country c
LEFT JOIN Match m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY c.name;''', engine)

* Create 3 `CASE WHEN` statements counting the matches played in each country across the 3 seasons.
* `END` your `CASE` statement without an `ELSE` clause.

In [34]:
pd.read_sql_query('''
SELECT c.name AS country,
       COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
	   COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
	   COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
FROM country c
LEFT JOIN match m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;''', engine)

### **COUNT and CASE WHEN with multiple conditions**

In Python, you have the ability to calculate a `SUM` of logical values (i.e., `TRUE`/`FALSE`) directly. In SQL, you have to convert these values into 1 and 0 before calculating a sum. This can be done using a `CASE` statement.

There's one key difference when using `SUM` to aggregate logical values compared to using `COUNT` in the previous exercise.

Your goal here is to use the `Country` and `Match` table to determine the *total number of matches won by the home team in each country* during the 2012/2013, 2013/2014, and 2014/2015 seasons.

* Create 3 `CASE` statements to "count" matches in the `'2012/2013'`, `'2013/2014'`, and `'2014/2015'` seasons, respectively.
* Have each `CASE` statement return a `1` for every match you want to include, and a `0` for every match to exclude.
* Wrap the `CASE` statement in a `SUM` to return the total matches played in each season.
* Group the query by the country name alias.

In [35]:
pd.read_sql_query('''
SELECT name AS country,
	   SUM(CASE WHEN season = '2012/2013' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2012_2013,
 	   SUM(CASE WHEN season = '2013/2014' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2013_2014,
       SUM(CASE WHEN season = '2014/2015' AND home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS matches_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

### **Calculating percent with CASE and AVG**

`CASE` statements will return any value you specify in your `THEN` clause. This is an incredibly powerful tool for robust calculations and data manipulation when used in conjunction with an aggregate statement. One key task you can perform is using `CASE` inside an `AVG` function to calculate a percentage of information in your database.

Here's an example of how you set that up:

```
AVG(CASE WHEN condition_is_met THEN 1
         WHEN condition_is_not_met THEN 0 END)
```

With this approach, it's important to accurately specify which records count as `0`, otherwise your calculations may not be correct!



Your task is to examine the number of wins, losses, and ties in each country. The `Match`es table is filtered to include all matches from the 2013/2014 and 2014/2015 seasons.

* Create 3 `CASE` statements to `COUNT` the total number of home team wins, away team wins, and ties, which will allow you to examine the total number of records.

In [36]:
pd.read_sql_query('''
SELECT name AS country, 
COUNT(CASE WHEN home_team_goal > away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'You can type' END) AS home_wins,
COUNT(CASE WHEN home_team_goal < away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'whatever for' END) AS away_wins,
COUNT(CASE WHEN home_team_goal = away_team_goal AND (season = '2013/2014' OR season = '2014/2015') THEN 'COUNT CASE'   END) AS ties
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

* Calculate the percentage of matches tied using a `CASE` statement inside `AVG`.
* Fill in the logical operators for each statement. Alias your columns as `ties_2013_2014` and `ties_2014_2015`, respectively.

In [37]:
pd.read_sql_query('''
SELECT name AS country,
AVG(CASE WHEN season='2013/2014' AND home_team_goal =  away_team_goal THEN 1 
		 WHEN season='2013/2014' AND home_team_goal <>  away_team_goal THEN 0 END) AS ties_2013_2014,
AVG(CASE WHEN season='2014/2015' AND home_team_goal =  away_team_goal  THEN 1 
		 WHEN season='2014/2015' AND home_team_goal <>  away_team_goal THEN 0 END) AS ties_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

* The previous "ties" columns returned values with 14 decimal points, which is not easy to interpret. Use the `ROUND` function to round to 2 decimal points.

In [38]:
pd.read_sql_query('''
SELECT name AS country,
ROUND(AVG(CASE WHEN season='2013/2014' AND home_team_goal =  away_team_goal THEN 1 
		 WHEN season='2013/2014' AND home_team_goal <>  away_team_goal THEN 0 END), 2) AS ties_2013_2014,
ROUND(AVG(CASE WHEN season='2014/2015' AND home_team_goal =  away_team_goal  THEN 1 
		 WHEN season='2014/2015' AND home_team_goal <>  away_team_goal THEN 0 END), 2) AS ties_2014_2015
FROM Country
LEFT JOIN Match
ON Country.id = Match.country_id
GROUP BY country;''', engine)

# **Simple Subqueries**

* Can be queried independently from the outer query
* Executed first before the main query

In [39]:
pd.read_sql_query(''' 
SELECT home_team_goal
FROM Match
WHERE home_team_goal > 
      (SELECT AVG(home_team_goal) FROM Match)
LIMIT 5;''', engine)

## **Subqueries in WHERE**

* สมมติอยากทราบว่า ในฤดูกาล 2012/2013 มีการแข่งขันกี่คู่ที่มี `home_team_goal > away_team_goal`

In [40]:
pd.read_sql_query('''
SELECT date, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal
FROM Match
WHERE season = '2012/2013'
AND home_team_goal > (SELECT AVG(home_team_goal) FROM Match)
LIMIT 10; ''', engine)

## **Subqueries filtering list with IN**

* ทีมไหนมาจากประเทศโปแลนด์?

In [41]:
display(pd.read_sql_query("SELECT * FROM Country WHERE name = 'Poland'", engine))
pd.read_sql_query('''SELECT team_long_name, team_short_name AS abbr
                     FROM Team 
                     WHERE team_api_id IN (SELECT home_team_api_id FROM Match WHERE country_id = 15722)
                     ORDER BY abbr;''', engine)

In [42]:
# เขียนด้วย INNER JOIN ก็พอไหวอยู่
pd.read_sql_query(''' 
SELECT DISTINCT team_long_name, team_short_name AS abbr
FROM Team
INNER JOIN Match ON (Team.team_api_id = Match.home_team_api_id) OR (Team.team_api_id = Match.away_team_api_id)
INNER JOIN Country ON Country.id = Match.country_id
WHERE name = 'Poland'
ORDER BY abbr''', engine)

### **Filtering using scalar subqueries**

Subqueries are incredibly powerful for performing complex filters and transformations. You can filter data based on single, *scalar* values using a subquery in ways you cannot by using `WHERE` statements or joins. Subqueries can also be used for more advanced manipulation of your data set. You will likely encounter subqueries in any real-world setting that uses relational databases.

In this exercise, you will generate a list of matches where the total goals scored (for both teams in total) is more than 3 times the average for games in the `Match` table, which includes only games played in the 2013/2014 season.

* Calculate *triple *the average home + away goals scored across all matches. This will become your subquery in the next step. Note that this column does not have an alias, so it will be called `?column?` in your results.

In [43]:
pd.read_sql('''SELECT 3 * AVG(home_team_goal + away_team_goal) 
               FROM Match 
               WHERE season = '2013/2014';''', engine)

* Select the `date`, `home_team_goal`, and `away_team_goal` in the main query.
* Filter the main query for matches where the total goals scored exceed the value in the subquery.

In [44]:
pd.read_sql(''' 
SELECT date, home_team_goal, away_team_goal
FROM Match
WHERE season = '2013/2014'
AND home_team_goal + away_team_goal > (SELECT 3 * AVG(home_team_goal + away_team_goal) 
                                       FROM Match 
                                       WHERE season = '2013/2014');''', engine)

### **Filtering using a subquery with a list**

Your goal in this exercise is to generate a list of teams that never played a game in their home city. Using a subquery, you will generate a list of unique `home_team_api_id` values from the unfiltered `?atch` table to exclude in the team table's `team_api_id` column.

In addition to filtering using a single-value (scalar) subquery, you can create a list of values in a subquery to filter data based on a complex set of conditions. This type of subquery generates <ins>a one column reference list</ins> for the main query. As long as the values in your list match a column in your main query's table, you don't need to use a join -- even if the list is from a separate table.

* Create a subquery in the `WHERE` clause that retrieves all unique `home_team_api_id` values from the `Match` table.
* Select the `team_long_name` and `team_short_name` from the `Team` table. Exclude all values from the subquery in the main query.

In [45]:
pd.read_sql_query('''
SELECT team_long_name,
       team_short_name
FROM Team 
WHERE team_api_id NOT IN (1601,1773,1957,2033,2182,2183,2186,4064,4087,6269,6403,6413,6433,6547,6631,7730,7788,7794,7819,7841,7842,7844,7869,7878,7943,7955,8019,8020,8021,8023,8024,8025,8027,8028,8030,8033,8066,8165,8177,8178,8191,8194,8197,8203,8226,8244,8245,8277,8284,8302,8305,8315,8342,8344,8348,8350,8357,8358,8370,8371,8372,8394,8406,8426,8429,8455,8456,8457,8460,8464,8466,8467,8472,8475,8481,8485,8521,8524,8525,8526,8528,8529,8530,8533,8534,8535,8537,8540,8543,8548,8550,8551,8558,8559,8560,8564,8569,8571,8573,8576,8581,8583,8586,8588,8592,8593,8597,8600,8602,8603,8611,8613,8633,8634,8635,8636,8639,8640,8649,8650,8654,8655,8659,8661,8667,8668,8673,8674,8686,8689,8696,8697,8721,8722,9747,9748,9761,9764,9768,9771,9772,9773,9776,9777,9783,9788,9789,9790,9791,9798,9800,9803,9804,9807,9810,9823,9824,9825,9826,9827,9829,9830,9831,9836,9837,9839,9847,9850,9851,9853,9857,9860,9864,9865,9869,9873,9874,9875,9876,9878,9879,9880,9882,9885,9888,9904,9905,9906,9908,9910,9925,9927,9930,9931,9938,9941,9956,9984,9985,9986,9987,9989,9991,9993,9994,9997,9998,10000,10001,10003,10167,10172,10179,10189,10190,10191,10192,10194,10199,10205,10211,10212,10214,10215,10217,10218,10219,10228,10229,10233,10235,10238,10242,10243,10249,10251,10252,10260,10261,10264,10265,10267,10268,10269,10281,158085,274581
);''', engine)

In [46]:
# ข้อมูลจริง ทำไมได้แบบนี้ สงสัยเป็นเพราะตาราง Match บน Kaggle กับ matches บน datacamp มีจำนวนแถวที่ต่างกันเยอะมาก
pd.read_sql_query('''
SELECT team_long_name,
       team_short_name
FROM Team 
WHERE team_api_id NOT IN (SELECT DISTINCT home_team_api_id FROM match);''', engine)

### **Filtering with more complex subquery conditions**
In the previous exercise, you generated a list of teams that have no home matches listed in the soccer database using a subquery in `WHERE`. Let's do some further exploration in this database by creating a list of teams that scored *8 or more goals in a home match*.

In order to do this, you will construct a subquery in the `WHERE` statement with its own filtering condition.

* Create a subquery in `WHERE` clause that retrieves all `home_team_api_id` values from `Match` with a `home_team_goal` score greater than or equal to `8`.
* Select the `team_long_name` and `team_short_name` from the `Team` table. Include all values from the subquery in the main query.

### * `season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')` ถูกใส่มาเพื่อให้ข้อมูล Match ใน Kaggle ตรงกับบน Datacamp

In [47]:
pd.read_sql_query('''
SELECT 	team_long_name,team_short_name
FROM Team
WHERE team_api_id IN (SELECT home_team_api_id
       				  FROM Match
       				  WHERE home_team_goal >= 8
                      AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'));''', engine)

# **Subqueries in FROM**

* เหมือนกับการเลือกข้อมูลจากตารางใหม่ แต่ตารางนั้นถูกสร้างมาภายใต้คำสั่งที่เขียนใน subquery
* ตัวอย่างเช่น สมมติต้องการหาทีมี่ได้ highest average home goals.

## **FROM Subqueries**

In [48]:
pd.read_sql_query('''
SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
FROM Match
LEFT JOIN Team
ON Match.home_team_api_id = Team.team_api_id
WHERE season = '2011/2012'
GROUP BY Team
LIMIT 5;''', engine)

ตัวอย่างไม่ค่อยดี เขียนปกติ ไม่ต้องทำเป็น subquery ก็ได้

In [49]:
pd.read_sql_query('''
SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
FROM Match
LEFT JOIN Team
ON Match.home_team_api_id = Team.team_api_id
WHERE season = '2011/2012'
GROUP BY Team
ORDER BY home_avg DESC
LIMIT 5;''', engine)

ถ้าจะเขียนเป็น subquery ก็ต้องระวัง
* ใส่ subquery ในวงเล็บ
* เอาเครื่องหมาย ; ออกจาก subquery
* จะใส่ alias หรือไม่ใส่ให้ subquery ก็ได้เหมือนกัน


In [50]:
pd.read_sql_query('''
SELECT team, home_avg
FROM (SELECT team_long_name AS team, AVG(home_team_goal) AS home_avg
      FROM Match
      LEFT JOIN Team
      ON Match.home_team_api_id = Team.team_api_id
      WHERE season = '2011/2012'
      GROUP BY Team)
ORDER BY home_avg DESC
LIMIT 3;''', engine)

## **Things to remember using subqueries in FROM**

* You can create multiple subqueries in one `FROM` statement -- กรณีนี้ ต้องใช้ Alias
* อาจจะต้อง join

### **Joining Subqueries in FROM**

The `Match` table in the European Soccer Database does not contain country or team names. You can get this information by joining it to the `Country` table, and use this to aggregate information, such as the number of matches played in each country.

If you're interested in filtering data from one of these tables, you can also create a subquery from one of the tables, and then join it to an existing table in the database. A subquery in `FROM` is an effective way of answering detailed questions that requires filtering or transforming data before including it in your final results.

Your goal in this exercise is to generate a subquery using the `Match` table, and then join that subquery to the `Country` table to calculate information about matches with 10 or more goals in total!

* Create the subquery to be used in the next step, which selects the `country_id` and match ID (`id`) from the `Match` table.
* Filter the query for matches with greater than or equal to `10` goals.

In [51]:
pd.read_sql_query('''
SELECT 
	-- Select the country ID and match ID
	country_id, 
    id 
FROM Match
-- Filter for matches with 10 or more goals in total
WHERE (home_team_goal + away_team_goal) >= 10
AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012');''', engine)

* Construct a subquery that selects only matches with `10` or more total goals.
* `INNER JOIN` the subquery onto `Country` in the main query.
* Select `country_name` from `Country` and count the `id` column from `Match`.

In [52]:
pd.read_sql_query('''
SELECT name AS country_name, COUNT(sub.id) AS matches
FROM Country
INNER JOIN (SELECT country_id, id 
            FROM Match
            WHERE (home_team_goal+away_team_goal) >= 10
            AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
ON Country.id = sub.country_id
GROUP BY name;''', engine)

จริงๆ แค่ `INNER JOIN` เฉยๆ ก็ได้ ไม่ต้องเขียนให้เป็น subquery

In [53]:
pd.read_sql_query('''
SELECT name AS country_name, COUNT(Match.id) AS matches
FROM Country
INNER JOIN Match
ON Country.id = Match.country_id
WHERE home_team_goal+away_team_goal >= 10
AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY name;''', engine)

### **Building on Subqueries in FROM**

In the previous exercise, you found that England, Netherlands, Germany and Spain were the only countries that had matches in the database where 10 or more goals were scored overall. Let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home versus away goals.

You'll notice that in this exercise, the table <ins>alias is excluded for every column selected in the main query</ins>. This is because the *main query* is extracting data from the *subquery*, which is treated as a single table.

* Complete the subquery inside the `FROM` clause. Select the country `name` from the `Country` table, along with the `date`, `the home_team_goal`, the `away_team_goal`, and the total goals columns from the `Match` table.
* Create a column in the subquery that adds home and away goals, called `total_goals`. This will be used to filter the main query.
* Select the country, date, home goals, and away goals in the main query.
* Filter the main query for games with 10 or more total goals.

In [54]:
pd.read_sql_query('''
SELECT country, date, home_team_goal, away_team_goal
FROM (SELECT c.name AS country, 
     	     m.date, 
     		 m.home_team_goal, 
     		 m.away_team_goal,
             m.home_team_goal + m.away_team_goal AS total_goals
      FROM Match m
      LEFT JOIN Country c
      ON m.country_id = c.id
      AND season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS subq
WHERE total_goals >= 10
AND country IS NOT NULL;''', engine)

# **Subqueries in SELECT**

* เพราะว่า เราไม่สามารถดึงค่า aggregated values ของ ungrouped query ได้ จึงต้องใช้ subquery in `SELECT`
* Returns a **<ins>single AGGREGATED value</ins>**
    * Include aggregate values to compare individuals values
    
สมมติอยากทราบว่า มีการแข่งขันในแต่ละฤดูกาลเป็นเท่าไร เมื่อเปรียบเทียบกับจำนวนการแข่งขันทั้งหมด
   * อย่าลืมว่า ข้อมูลไม่เท่ากันกับบนเว็บดาต้าแคมพ์

In [55]:
display(pd.read_sql("SELECT COUNT(id) FROM Match", engine))
print("\n")
display(pd.read_sql("SELECT COUNT(id) FROM Match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')", engine))

In [56]:
pd.read_sql_query(''' 
SELECT season, COUNT(id) AS matches, 12837 AS total_matches
FROM Match
WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY season''', engine)

ถ้าใช้ subquery

In [57]:
pd.read_sql_query(''' 
SELECT season, COUNT(id) AS matches, 
    (SELECT COUNT(id) FROM Match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS total_matches
FROM Match
WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')
GROUP BY season''', engine)

## **SELECT subquries for mathematical operations**

In [58]:
pd.read_sql_query('''SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) FROM Match WHERE season = '2011/2012'; ''', engine)

สมมติ ต้องการคำนวณว่าในการแข่งขันแต่ละครั้งนั้น จำนวนประตูที่ทำได้มีความแตกต่างจากจำนวนประตูเฉลี่ยในฤดูกาล 2011/2012 อย่างไร
* คือจะคำนวณตัวเลข `2.72` ไว้ก่อนก็ได้ หรือจะใช้เป็น subquery ตามด้านล่าง

In [59]:
pd.read_sql_query(''' 
SELECT date,
       home_team_goal + away_team_goal AS goals,
       home_team_goal + away_team_goal - (SELECT AVG(home_team_goal + away_team_goal) 
                                          FROM Match 
                                          WHERE season = '2011/2012') AS diff
FROM Match
WHERE season = '2011/2012'
LIMIT 4; ''', engine)

## **จงจำไว้ว่า**

* subqueries ใน `SELECT` statement จะต้องให้ค่าข้อมูลแค่ค่าเดียว ต่อ 1 แถว  <ins>และเป็นค่า aggregated value</ins>
* เพราะข้อมูลใน subquery จะเอาไปใช้ต่อกับทุกๆแถว
* ### เงื่อนไขใน `WHERE` statement จะต้องเหมือนกันทั้งใน subquery และ main query

### **Add a subquery to the SELECT clause**

Subqueries in `SELECT` statements generate a single value that allow you to pass an aggregate value down a data frame. This is useful for performing calculations on data within your database.

In the following exercise, you will construct a query that calculates the *average number of goals per match in each country's league*.

* In the subquery, select the average total goals by adding `home_team_goal` and `away_team_goal`.
* Filter the results so that only the average of goals in the `2013/2014` season is calculated.
* In the main query, select the average total goals by adding `home_team_goal` and `away_team_goal`. This calculates the average goals for each league.
* Filter the results in the main query the same way you filtered the subquery. Group the query by the league name.

In [60]:
pd.read_sql_query('''
SELECT name AS league,
    -- Select and round the league's total goals
       ROUND(AVG(home_team_goal + away_team_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
       (SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) FROM match WHERE season = '2013/2014') AS overall_avg
FROM League
LEFT JOIN Match
USING (country_id)
-- Filter for the 2013/2014 season ตรงนี้ต้องเหมือนกันกับใน subquery
WHERE season = '2013/2014'
GROUP BY League;''', engine)

### **Subqueries in Select for Calculations**

Subqueries in `SELECT` are a useful way to create calculated columns in a query. A subquery in `SELECT` can be treated as a <ins>single numeric value</ins> to use in your calculations. When writing queries in `SELECT`, it's important to remember that filtering the main query does not filter the subquery -- and vice versa.

In the previous exercise, you created a column to compare each league's average total goals to the overall average goals in the `2013/2014` season. In this exercise, you will add a column that directly compares these values by subtracting the overall average from the subquery.

* Select the average goals scored in a match for each league in the main query.
* Select the average goals scored in a match overall for the `2013/2014` season in the subquery.
* Subtract the subquery from the average number of goals calculated for each league.
* Filter the main query so that only games from the `2013/2014` season are included.

In [61]:
pd.read_sql_query('''
SELECT name AS league,
	   ROUND(AVG(home_team_goal + away_team_goal),2) AS avg_goals,
       ROUND(AVG(home_team_goal + away_team_goal) - (SELECT AVG(home_team_goal + away_team_goal) FROM Match WHERE season = '2013/2014'), 2) AS diff
FROM League
LEFT JOIN Match
USING(country_id)
WHERE season = '2013/2014'
GROUP BY league;''', engine)

# **Subqueries everywhere! And best practices!**



In [62]:
pd.read_sql_query('''
SELECT country_id, name,
       ROUND(AVG(home_team_goal + away_team_goal),2) AS avg_goals,
      (SELECT ROUND(AVG(home_team_goal + away_team_goal),2) FROM Match WHERE season = '2013/2014') AS overall_avg
FROM  (SELECT country_id, home_team_goal, away_team_goal, season FROM Match WHERE home_team_goal > 5) AS matches
INNER JOIN Country ON Country.id = matches.country_id
WHERE season = '2013/2014'
GROUP BY country_id, name
HAVING (AVG(home_team_goal + away_team_goal) > (SELECT AVG(home_team_goal + away_team_goal) FROM Match WHERE season = '2013/2014'));''', engine)

### **ALL the subqueries EVERYWHERE**

In soccer leagues, games are played at *different <ins>stages</ins>*. Winning teams progress from one stage to the next, until they reach the final stage. In each stage, the stakes become higher than the previous one. The `Match` table includes data about the different stages that each match took place in.

In this lesson, you will build a final query across 3 exercises that will contain three subqueries -- one in the `SELECT` clause, one in the `FROM` clause, and one in the `WHERE` clause. In the final exercise, your query will extract data examining the average goals scored in each stage of a match. Does the average number of goals scored change as the stakes get higher from one stage to the next?

* Extract the average number of `home_team_goal` + `away_team_goal` in two `SELECT` subqueries.
* Calculate the average `home_team_goal` +`away_team_goal` for the specific `stage` in the main query.
* Filter both subqueries and the main query so that only data from the `2012/2013` season is included.
* Group the query by the `stage` column.

In [63]:
pd.read_sql_query('''
SELECT 
  -- Select the stage and average goals for each stage
	stage,
    ROUND(AVG(home_team_goal + away_team_goal), 2) AS avg_goals,
  -- Select the average overall goals for the 2012/2013  season
    ROUND((SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013'), 2) AS overall
FROM match
-- Filter for the 2012/2013 season
WHERE season = '2012/2013'
-- Group by stage
GROUP BY stage;''', engine)

### **Add a subquery in FROM**
In the previous exercise, you created a data set listing the average home and away goals in each match stage of the 2012/2013 match season.

In this next step, you will turn the main query into a *subquery* to extract a list of stages where the average home goals in a stage is *higher than* the overall average for home goals in a match.

* Calculate the average home goals and average away goals from the match table for each stage in the `FROM` clause subquery.
* Add a subquery to the `WHERE` clause that calculates the overall average home goals.
* Filter the main query for stages where the average home goals is higher than the overall average.
* Select the `stage` and `avg_goals` columns from the subquery into the main query.

In [64]:
pd.read_sql_query('''
SELECT 
	-- Select the stage and average goals from the subquery
	stage,
	ROUND(avg_goals, 2) AS avg_goals
FROM 
	-- Select the stage and average goals in 2012/2013
	(SELECT stage, AVG(home_team_goal + away_team_goal) AS avg_goals 
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage) 
WHERE 
	-- where the average home goals in a stage is higher than the overall average for home goals in a match
	avg_goals > (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013');''', engine)

### **Add a subquery in SELECT**

In the previous exercise, you added a subquery to the `FROM` statement and selected the stages where the number of average goals in a stage exceeded the overall average number of goals in the 2012/2013 match season. In this final step, you will add a subquery in `SELECT` to compare the average number of goals scored in each stage to the total.

* Create a subquery in `SELECT` that yields the average goals scored in the `2012/2013` season. Name the new column `overall_avg`.
* Create a subquery in `FROM` that calculates the average goals scored in each stage during the `2012/2013` season.
* Filter the main query for stages where the average goals exceeds the overall average in `2012/2013`.

In [65]:
pd.read_sql_query('''
-- Select the stage and average goals from s
SELECT stage,
       ROUND(avg_goals, 2) AS avg_goal,
-- Select the overall average for 2012/2013
       (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
	-- Select the stage and average goals in 2012/2013 from match
	(SELECT stage,
            AVG(home_team_goal + away_team_goal) AS avg_goals
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage)
WHERE 
	-- Filter the main query using the subquery :: ระวัง ตัว s ข้างหลัง avg_goal
	avg_goals > (SELECT AVG(home_team_goal + away_team_goal) 
                    FROM match WHERE season = '2012/2013');''' , engine)

# **Correlated Subquiries**

## **จากตัวอย่างที่แล้ว** 
### Which match stages tend to have a higher than average number of goals scored?

In [66]:
pd.read_sql_query('''
SELECT stage,
       ROUND(avg_goals, 2) AS avg_goal,
      (SELECT AVG(home_team_goal + away_team_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM  
      (SELECT stage,
              AVG(home_team_goal + away_team_goal) AS avg_goals
	   FROM match
	   WHERE season = '2012/2013'
	   GROUP BY stage)
WHERE avg_goals > (SELECT AVG(home_team_goal + away_team_goal) 
                   FROM match WHERE season = '2012/2013');''' , engine)

* ผลลัพธ์ข้างบนนี้ สามารถทำได้ด้วยการเขียน correlated subquiries ????

In [67]:
pd.read_sql_query('''
SELECT s.stage,
       ROUND(s.avg_goals,2) AS avg_goal,
       (SELECT AVG(home_team_goal + away_team_goal)
       FROM match
       WHERE season = '2012/2013') AS overall_avg
FROM
       (SELECT stage,
               AVG(home_team_goal + away_team_goal) AS avg_goals
        FROM match
        WHERE season = '2012/2013'
        GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_team_goal + away_team_goal)
                     FROM match AS m
                     WHERE s.stage > m.stage);''' , engine)

In [68]:
pd.read_sql_query('''
SELECT s.stage,
       ROUND(s.avg_goals,2) AS avg_goal,
       (SELECT AVG(home_team_goal + away_team_goal)
        FROM match
        WHERE season = '2012/2013') AS overall_avg
FROM
       (SELECT stage,
               AVG(home_team_goal + away_team_goal) AS avg_goals
        FROM match
        WHERE season = '2012/2013'
        GROUP BY stage) AS s
WHERE s.avg_goals > (SELECT AVG(home_team_goal + away_team_goal)
                     FROM match AS m
                     WHERE s.stage < m.stage);''' , engine)

### Have no idea why the result is incorrect.  

* The inequality value in the last subquery is somewhat strange.  When I copied the code from slides and ran it on Kaggle, it returned incorrect values.  When I changed it to "`s.stage < m.stage`", the result is closer to the uncorrelated subqueries, but still incorrect (Look at stage `15` for example.).  
* ### Perhaps because the correlated subqueries are evalueated in loops -- once for each row geneerated by the dataset
* ลองมาดูตัวอย่างที่ง่ายขึ้น สมมติอยากทราบว่า ในแต่ละประเทศ มีการทำประตูเฉลี่ยเท่าไหร่?
    * ถ้าใช้ `LEFT JOIN` เขียน ก็จะได้แบบด้านล่างนี้

In [69]:
pd.read_sql_query(''' 
SELECT name AS country,
       AVG(home_team_goal + away_team_goal) AS avg_goals
FROM country  c
LEFT JOIN match m ON c.id = m.country_id
GROUP BY country;''', engine)

## **เขียนแบบ CORRELATED SUBQUERY**

In [70]:
pd.read_sql_query('''
SELECT name AS country,
      (SELECT AVG(home_team_goal + away_team_goal) FROM match m
       WHERE m.country_id = c.id) AS avg_goals
FROM country c;''', engine)

# เขียน correlated subquery แล้ว ไม่จำเป็นต้องมี GROUP BY ต่อท้าย

### **Basic Correlated Subqueries**

Correlated subqueries are subqueries that reference one or more columns in the main query. Correlated subqueries depend on information in the main query to run, and thus, <ins>cannot be executed on their own</ins>.

Correlated subqueries are evaluated in SQL <ins>once per row of data retrieved</ins> -- a process that takes a lot more computing power and time than a simple subquery.

In this exercise, you will practice using correlated subqueries to examine matches with scores that are extreme outliers for each country -- *above 3 times the average score*!

* Select the `country_id`, `date`, `home_team_goal`, and `away_team_goal` columns in the main query.
* Complete the `AVG` value in the subquery.
* Complete the subquery column references, so that `country_id` is matched in the main and subquery.

### Correlated Subqueries ช้ามาก ใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
```


### แทน `match` เพราะว่า ตรงกับจำนวนตัวอย่างที่ใช้ในดาต้าแคมพ์ รันเต็มๆ ครึ่งชั่วโมงก็ยังไม่ออก



In [71]:
pd.read_sql('''
SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal
-- ใช้แทน match ตรงนี้
FROM (SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS main
WHERE home_team_goal + away_team_goal > (SELECT AVG((sub.home_team_goal + sub.away_team_goal) * 3)
                                         -- ใช้แทน match ตรงนี้
                                         FROM (SELECT country_id,
                                               date,
                                               home_team_goal, 
                                               away_team_goal, 
                                               season FROM match 
                                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
                                         -- Join the main query to the subquery in WHERE
                                         WHERE main.country_id = sub.country_id);''', engine)

### **Correlated subquery with multiple conditions**

Correlated subqueries are useful for <ins>matching data across multiple columns</ins>. In the previous exercise, you generated a list of matches with extremely high scores for each country. In this exercise, you're going to add an additional column for matching to answer the question -- *what was the highest scoring match for each country, in each season?*

*Note: this query may take a while to load.

* Select the `country_id`, `date`, `home_team_goal`, and `away_team_goal` columns in the main query.
* Complete the subquery: Select the matches with the highest number of total goals.
* Match the subquery to the main query using `country_id` and `season`.
* Fill in the correct logical operator so that total goals equals the max goals recorded in the subquery.

### Correlated Subqueries ช้ามาก ใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
```


### แทน `match` เพราะว่า ตรงกับจำนวนตัวอย่างที่ใช้ในดาต้าแคมพ์ รันเต็มๆ ครึ่งชั่วโมงก็ยังไม่ออก

In [72]:
pd.read_sql('''
SELECT country_id, date, home_team_goal, away_team_goal   
FROM (SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal,
       season
       FROM match 
       WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	   home_team_goal + away_team_goal = (SELECT MAX(sub.home_team_goal + sub.away_team_goal)
                                          FROM (SELECT country_id,
                                                date,
                                                home_team_goal, 
                                                away_team_goal,
                                                season
                                                FROM match 
                                                WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) AS sub
                                          WHERE main.country_id = sub.country_id
                                          AND main.season = sub.season);''', engine)

# **Nested Subqueries**

* จริงๆก็ทำไปแล้ว เพราะใช้

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 ตัวอย่างด้านล่าง How much did each country's average differ from the overall average?
 

In [73]:
pd.read_sql_query(''' 
SELECT name AS country,
       ROUND(AVG(home_team_goal + away_team_goal), 4) AS avg_goals,
       ROUND(AVG(home_team_goal + away_team_goal) - (SELECT AVG(home_team_goal + away_team_goal) FROM match),4) AS avg_ddiff
FROM country  c
LEFT JOIN match m ON c.id = m.country_id
GROUP BY country;''', engine)

## **Inside a subquery**

* ในแต่ละเดือน จำนวนประตูทั้งหมดที่ทำได้ แตกต่างจากจำนวนประตูทั้งหมดโดยเฉลี่ยที่ทำได้อย่างไร

## **Inner Subquery**

ใช้ `EXTRACT(MONTH from date)` ไม่ได้ใน SQLite ต้องใช้ `strftime('%m', date)` แทน

In [74]:
pd.read_sql_query('''
SELECT strftime('%m', date) AS month,
       SUM(home_team_goal + away_team_goal) AS goals
FROM match
GROUP BY month;''', engine)

## **Outer subquery**

หา average ของ sum

In [75]:
pd.read_sql_query('''
SELECT AVG(goals)
FROM (SELECT strftime('%m', date) AS month,
             SUM(home_team_goal + away_team_goal) AS goals
      FROM match
      GROUP BY month)''', engine)


## **Final query**

In [76]:
pd.read_sql_query('''
SELECT strftime('%m', date) AS month,
       SUM(home_team_goal + away_team_goal) AS total_goals,
       SUM(home_team_goal + away_team_goal) - (SELECT AVG(goals)
                                               FROM (SELECT strftime('%m', date) AS month,
                                                            SUM(home_team_goal + away_team_goal) AS goals
                                                     FROM match
                                                     GROUP BY month)) AS diff
FROM match
GROUP BY month
LIMIT 4;''', engine)

# **Correlated Nested Subqueries**

* จำนวนประตูเฉลี่ยที่ทำได้ของแต่ละประเทศในฤดูกาลที่ `2011/2012` เป็นอย่างไร

In [77]:
pd.read_sql_query('''
SELECT name AS country,
      (SELECT AVG(home_team_goal + away_team_goal)
       FROM match m
-- Correlates with main query
       WHERE m.country_id = c.id 
-- Begin inner subquery
       AND id IN (SELECT id 
                  FROM match
                  WHERE season = '2011/2012')) AS avg_goals
FROM country c
GROUP BY country;''', engine)

## ตัวอย่างไม่เหมาะสม เพราะสามารถใช้ LEFT JOIN แบบง่ายๆได้เลย

In [78]:
pd.read_sql_query(''' 
SELECT name AS country,
       AVG(home_team_goal + away_team_goal) AS avg_goals
FROM country c
LEFT JOIN match m ON m.country_id = c.id 
WHERE season = '2011/2012'
GROUP BY country;''', engine)

### **Nested simple subqueries**

Nested subqueries can be either simple or correlated.

Just like an unnested subquery, a nested subquery's components can be executed independently of the outer query, while a correlated subquery requires both the outer and inner subquery to run and produce results.

In this exercise, you will practice creating a nested subquery to examine *the highest total number of goals in each season, overall, and during July across all seasons*.

* Complete the main query to select the `season` and the  `MAX(home_team_goal + away_team_goal)` in a match for each season. Name this `max_goals`.
* Complete the first simple subquery to select the `MAX(home_team_goal + away_team_goal)` in a `match` across all seasons. Name this `overall_max_goals`.
* Complete the nested subquery to select the `MAX(home_team_goal + away_team_goal)` in a `match` played in July across all seasons.
* Select the `MAX(home_team_goal + away_team_goal)` in the outer subquery. Name this entire subquery `july_max_goals`.

### คราวนี้ ไม่ต้องใช้ 

```
(SELECT country_id,
       date,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 
### แทน `match` เพราะมันต้องใส่หลายที่มาก ผลลัพธ์ต่างกันนิดหน่อยช่างมัน

In [79]:
pd.read_sql_query('''
SELECT season,
       MAX(home_team_goal + away_team_goal) AS max_team_goals,
      (SELECT MAX(home_team_goal + away_team_goal) FROM match) AS overall_max_team_goals,
      (SELECT MAX(home_team_goal + away_team_goal) FROM match WHERE id IN (SELECT id 
                                                                           FROM match 
                                                                           WHERE strftime('%m', date) = '07')) AS july_max_team_goals
FROM match
GROUP BY season; ''', engine)

### **Nest a subquery in FROM**
What's the average number of matches per season where a team scored 5 or more goals? How does this differ by country?

Let's use a nested, correlated subquery to perform this operation. In the real world, you will probably find that nesting multiple subqueries is a task you don't have to perform often. In some cases, however, you may find yourself struggling to properly group by the column you want, or to calculate information requiring multiple mathematical transformations (i.e., an `AVG` of a `COUNT`).

Nesting subqueries and performing your transformations one step at a time, adding it to a subquery, and then performing the next set of transformations is often the easiest way to yield accurate information about your data. Let's get to it!

* `SELECT country_id, season, id`
* Generate a list of matches where ***at least one team*** scored ***5 or more goals***.

### ใช้ 

```
(SELECT country_id,
       id,
       home_team_goal, 
       away_team_goal, 
       season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
 ```
 
### แทน `match` 

In [80]:
pd.read_sql('''SELECT country_id, season, id 
FROM (SELECT country_id,
             id,
             home_team_goal, 
             away_team_goal, 
             season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
WHERE home_team_goal > 5 OR away_team_goal > 5;''', engine)

* Turn the query from the previous step into a subquery in the `FROM` statement.
* `COUNT` the match `id`s generated in the previous step, and group the query by `country_id` and `season`.

In [81]:
pd.read_sql_query(''' 
-- Count match ids
SELECT country_id,season, COUNT(id) AS matches
FROM (SELECT country_id, season, id
	  FROM (SELECT country_id,
                   id,
                   home_team_goal, 
                   away_team_goal, 
                   season 
            FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
	  WHERE home_team_goal >= 5 OR away_team_goal >= 5)
-- Group by country_id and season
GROUP BY country_id, season;''', engine)

* Finally, declare the same query from step 2 as a subquery in `FROM` with the alias `outer_s`.
* `LEFT JOIN` it to the `country` table using the outer query's `country_id` column.
* Calculate an `AVG` number of high scoring `matches` per country in the main query.

In [82]:
pd.read_sql_query('''
SELECT name AS country,
       ROUND(AVG(outer_s.matches), 2) AS avg_seasonal_high_scores
FROM country c
LEFT JOIN (SELECT country_id, season, COUNT(id) AS matches
           FROM (SELECT country_id, season, id
	             FROM (SELECT country_id,
                              id,
                              home_team_goal, 
                              away_team_goal, 
                              season 
                        FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')) 
	             WHERE home_team_goal >= 5 OR away_team_goal >= 5) AS inner_s
           GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country;''', engine)

# **COMMON TABLE EXPRESSIONS**

* คือการเอา subquery มาเขียนไว้ก่อนนำหน้าในรูปแบบของ CTE
* สมมติ ด้านล่างนี้ อยากรู้ว่า ถ้าจะคัดมาเฉพาะ match ที่มีการทำประตตูทั้งหมด มากกว่า 10

## เขียนในรูปแบบ subquery

In [83]:
pd.read_sql_query('''
SELECT name AS country,
       COUNT(s.id) AS matches
FROM country c
INNER JOIN (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) >= 10) AS s
ON c.id = s.country_id
GROUP BY country;''', engine)

## เขียน CTE แทน subquery

In [84]:
pd.read_sql_query('''
WITH s AS (SELECT country_id, id
           FROM match
           WHERE (home_team_goal + away_team_goal) >= 10) 

SELECT name AS country,
       COUNT(DISTINCT s.id) AS matches
FROM country c
INNER JOIN s
ON c.id = s.country_id
GROUP BY country;''', engine)

## **Multiple CTEs**

In [85]:
pd.read_sql_query('''
WITH s1 AS (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) >= 10),
    s2 AS  (SELECT country_id, id
            FROM match
            WHERE (home_team_goal + away_team_goal) <= 1)
            
SELECT name AS country,
       COUNT(DISTINCT s1.id) AS high_scores_matches,
       COUNT(DISTINCT s2.id) AS low_scores_matches
FROM country AS c
INNER JOIN s1 ON c.id = s1.country_id
INNER JOIN s2 ON c.id = s2.country_id
GROUP BY country;''', engine)

### **Clean up with CTEs**

Above, you generated a list of countries and the number of matches in each country with more than 10 total goals. The query in that exercise utilized a subquery in the `FROM` statement in order to filter the matches before counting them in the main query. Below is the query you created:

```
SELECT name AS country,
       COUNT(sub.id) AS matches
FROM country AS c
INNER JOIN (SELECT country_id, id 
            FROM match
            WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country;
```


You can list one (or more) subqueries as **common table expressions** (CTEs) by *declaring* them ahead of your main query, which is an excellent tool for organizing information and placing it in a logical order.

In this exercise, let's rewrite a similar query using a CTE.

* Complete the syntax to declare your CTE.
* Select the `country_id` and `match id` from the `match` table in your CTE.
* Left join the CTE to the league table using `country_id`.

query ข้างบนได้ตามนี้ (ด้านล่างเปลี่ยนจาก `country` เป้น `league` เพราะเข้ากับแบบฝึกหัดมากกว่า)

In [86]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_goal, 
                               away_team_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))

SELECT name AS league,
       COUNT(sub.id) AS matches
FROM league l
INNER JOIN (SELECT country_id, id 
            FROM filtered_match
            WHERE (home_team_goal + away_team_goal) >= 10) AS sub
ON l.id = sub.country_id
GROUP BY league
ORDER BY matches;''', engine)

In [87]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_goal, 
                               away_team_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),

    match_list AS (SELECT country_id, id
                   FROM filtered_match
                   WHERE (home_team_goal + away_team_goal) >= 10)

SELECT name AS league,
       COUNT(match_list.id) AS matches
FROM league l
LEFT JOIN match_list ON l.id = match_list.country_id
GROUP BY name
ORDER BY matches;''', engine)

### **Organizing with CTEs**

Previously, you modified a query based on a statement you completed in chapter 2 using common table expressions.

This time, let's expand on the exercise by looking at details about matches with very high scores using CTEs. Just like a subquery in `FROM`, you can join tables *inside* a CTE.

* Declare your CTE, where you create a list of all matches with the league name.
* Select the `league`, `date`, `home_team_goal`, and `away_team_goal` from the CTE.
* Filter the main query for matches with `10` or more goals.



In [88]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
    
         match_list AS (SELECT name AS league, 
     	                       date, 
  		                       home_goal, 
  		                       away_goal,
                               home_goal + away_goal AS total_goals
                        FROM filtered_match
                        LEFT JOIN league USING(country_id))

SELECT league, date, home_goal, away_goal
FROM match_list
WHERE total_goals >= 10;''', engine)

### **CTEs with nested subqueries**

If you find yourself listing multiple subqueries in the `FROM` clause with nested statement, your query will likely become long, complex, and difficult to read.

Since many queries are written with the intention of being saved and re-run in the future, proper organization is key to a seamless workflow. Arranging subqueries as CTEs will save you time, space, and confusion in the long run!

* Declare a CTE that calculates the total goals from matches in August of the `2013/2014` season.
* `LEFT JOIN` the CTE onto the `league` table using `country_id` from the `match_list` CTE.
* Filter the list on the inner subquery to only select matches in August of the `2013/2014` season.

In [89]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal, 
                               season FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
        match_list AS  (SELECT country_id, 
                               home_goal + away_goal AS goals
                               FROM filtered_match
  	-- Create a list of match IDs to filter data in the CTE
                               WHERE id IN (SELECT id FROM filtered_match
                                            WHERE season = '2013/2014' AND strftime('%m', date) = '08'))
-- Select the league name and average of goals in the CTE
SELECT name,
       ROUND(AVG(goals), 2)
FROM league
-- Join the CTE onto the league table
LEFT JOIN match_list USING(country_id)
GROUP BY name;''', engine)

# **Get team names with a subquery**

Let's solve a problem we've encountered a few times in this course so far  
*  *How do you get both the home and away team names into one final query result*?

Out of the 4 techniques we just discussed, this can be performed using subqueries, correlated subqueries, and CTEs. Let's practice creating similar result sets using each of these 3 methods over the next 3 exercises, starting with subqueries in `FROM`.

* Create a query that `LEFT JOIN` `team` to `match` in order to get the identity of the home team. This becomes the subquery in the next step.

In [90]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
SELECT m.id, team_long_name AS hometeam
FROM filtered_match m
LEFT JOIN team t
ON m.hometeam_id = t.team_api_id
LIMIT 5;''', engine)

* Add a second subquery to the `FROM` statement to get the `away_team_name`, changing only the `hometeam_id`. Left join both subqueries to the `match` table on the `id` column.
* Warning: if your code is timing out, you have probably made a mistake in the `JOIN` and tried to join on the wrong fields which caused the table to be too big! Check your `ON` conditions!

In [91]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
SELECT date,
    -- Get the home and away team names
       hometeam,
       awayteam,
       home_goal,
       away_goal
FROM filtered_match m
-- Join the home subquery to the match table
LEFT JOIN (SELECT filtered_match.id, team.team_long_name AS hometeam
           FROM filtered_match
           LEFT JOIN team ON filtered_match.hometeam_id = team.team_api_id) AS home
USING(id)
-- Join the away subquery to the match table
LEFT JOIN (SELECT filtered_match.id, team.team_long_name AS awayteam
           FROM filtered_match
           LEFT JOIN team ON filtered_match.awayteam_id = team.team_api_id) AS away
USING(id);''', engine)

### **Get team names with correlated subqueries**

Let's solve the same problem using correlated subqueries 
* *How do you get both the home and away team names into one final query result?*

This can easily be performed using correlated subqueries. But how might that impact the performance of your query? Complete the following steps and let's find out!

Please note that your query will run MUCH MUCH more slowly than the previous exercise!

* Using a correlated subquery in the `SELECT` statement, match the `team_api_id` column from `team` to the `hometeam_id` from `match`.

In [92]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date,
      (SELECT team_long_name
       FROM team t
-- Connect the team to the match table
       WHERE t.team_api_id = m.hometeam_id) AS hometeam
FROM filtered_match m;''', engine)



* Create a second correlated subquery in `SELECT`, yielding the away team's name.
* Select the home and away goal columns from `match` in the main query.

In [93]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date,
      (SELECT team_long_name
       FROM team AS t
       WHERE t.team_api_id = m.hometeam_id) AS hometeam,
-- Connect the team to the match table
      (SELECT team_long_name
       FROM team AS t
       WHERE t.team_api_id = m.awayteam_id) AS awayteam,
    -- Select home and away goals
     home_goal,
     away_goal
FROM filtered_match m;''', engine)

### **Get team names with CTEs**

You've now explored two methods for answering the question, 
* *How do you get both the home and away team names into one final query result?*

Let's explore the final method - common table expressions. Common table expressions are similar to the subquery method for generating results, mainly differing in syntax and the order in which information is processed.

* Select `id` from `match` and `team_long_name` from `team`. Join these two tables together on `hometeam_id` in `match` and `team_api_id` in `team`.

In [94]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, 
       team_long_name AS hometeam
FROM filtered_match m
-- Join team to match using team_api_id and hometeam_id
LEFT JOIN team t 
ON m.hometeam_id = t.team_api_id;''', engine)

* Declare the query from the previous step as a common table expression. `SELECT` everything from the CTE into the main query. ***Your results will not change at this step!***

In [95]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
               home AS (SELECT m.id, t.team_long_name AS hometeam
                        FROM filtered_match m
                        LEFT JOIN team t 
                        ON m.hometeam_id = t.team_api_id)
-- Select everything from home
SELECT *
FROM home;''', engine)

* Let's declare the second CTE, `away`. Join it to the first CTE on the `id` column.
* Add `date`, `home_goal`, and `away_goal` columns to the CTEs. `SELECT` them into the main query.

In [96]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
               home AS (SELECT m.id, date, team_long_name AS hometeam, home_goal
                        FROM filtered_match m
                        LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
-- Declare and set up the away CTE
               away AS (SELECT m.id, date, team_long_name AS awayteam, away_goal
                        FROM filtered_match m
                        LEFT JOIN team t ON m.awayteam_id = t.team_api_id)
-- Select date, home_goal, and away_goal
SELECT home.date,
       home.hometeam,
       away.awayteam,
       home.home_goal,
       away.away_goal
-- Join away and home on the id column
FROM home
INNER JOIN away
USING(id);''', engine)

## ว่ากันด้วยเรื่องของ GROUP BY

In [97]:
pd.read_sql_query('''
SELECT country_id, season, date, AVG(home_team_goal) AS avg_home
FROM match
GROUP BY country_id, season, date
Order BY country_id, season, date;''', engine)


### Query ด้านล่างรันบน SQL ปกติไม่ได้ จะเกิด error 

```
column "match.season" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT country_id, season, date, AVG(home_goal) AS avg_home
```

### ถึง SQLite จะรันได้ ก็ได้ผลออกมาผิด
* คอลัมน์ season และ date มีแค่ฤดูกาล `2008/2009`

## ต้องใช้ `WINDOW` Function!

In [98]:
pd.read_sql_query('''
SELECT country_id, season, date, AVG(home_team_goal) AS avg_home
FROM match
GROUP BY country_id
Order BY country_id, season, date;''', engine)

# **What is a Window function?**

* ย้อนกลับไปคำถาม *How many goals were scored in each match in 2011/2012 and how did that compare to the average?*



In [99]:
pd.read_sql_query('''SELECT date, home_team_goal + away_team_goal AS goals, 
                            (SELECT AVG(home_team_goal + away_team_goal)
                                    FROM match
                                    WHERE season = '2011/2012') AS overall_avg
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

### ผลลัพธ์ข้างบนสามารถทำได้ด้วยการใช้ `OVER` statement
* `OVER()` tells SQL to pass this aggregate value over the existing result set (`2011/2012`)

In [100]:
pd.read_sql_query('''SELECT date, home_team_goal + away_team_goal AS goals, 
                            AVG(home_team_goal + away_team_goal) OVER() AS overall_avg
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

# **Generate a Rank**
* ตามชื่อเลย คือ สร้างคอลัมน์ใหม่แล้วจัดอันดับให้คอลัมน์ที่เราระบุ
* *What is the rank of matches based on number of goals scored?*

In [101]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals 
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

### ใช้คำสั่ง `RANK() OVER(ORDER BY column) AS col_rank`

By default, `RANK()` orders from smallest to largest ถ้าจะเรียงกลับ ให้ใส่ `DESC`

ข้อควรจำ:
1. Window functions are processed <ins>***after the entire query***</ins> except the final `ORDER BY` statement.

In [102]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals, 
                             RANK() OVER(ORDER BY home_team_goal + away_team_goal) AS goals_rank
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

In [103]:
pd.read_sql_query(''' SELECT date, home_team_goal + away_team_goal AS goals, 
                             RANK() OVER(ORDER BY home_team_goal + away_team_goal DESC) AS goals_rank
                      FROM match 
                      WHERE season = '2011/2012'
                      LIMIT 4;''', engine)

### **The match is OVER**

The `OVER()` clause allows you to pass an aggregate function down a data set, similar to subqueries in `SELECT`. The `OVER()` clause offers significant benefits over subqueries in select -- namely, your queries will run faster, and the `OVER()` clause has a wide range of additional functions and clauses you can include with it.

In this exercise, you will revise some queries from previous chapters using the `OVER()` clause.

* Select the match `id`, country `name`, `season`, `home_goal`, and `away_goal` from the `filtered_match` CTE and `country` tables.
* Complete the query that calculates the average number of goals scored overall and then includes the aggregate value in each row ***using a window function***.

In [104]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, 
       name AS country, 
       season,
	   home_goal,
	   away_goal,
    -- Use a window to include the aggregate average in each row
	   AVG(home_goal + away_goal) OVER() AS overall_avg
FROM filtered_match m
LEFT JOIN country c ON m.country_id = c.id;''', engine)

### **What's OVER here?**

Window functions allow you to create a `RANK` of information according to any variable you want to use to sort your data. When setting this up, you will need to specify what column/calculation you want to use to calculate your rank. This is done by including an <ins>`ORDER BY`clause inside the `OVER()`</ins> clause. Below is an example:

```
SELECT id,
       RANK() OVER(ORDER BY home_goal) AS rank
FROM match;
```

In this exercise, you will create a data set of ranked matches according to which leagues, on average, score the most goals in a match.


* Select the league `name` and average total goals scored from `league` and `filtered_match` CTE.
* Complete the window function so it calculates the rank of average goals scored across all leagues in the database.
* Order the rank by the average total of home and away goals scored.

In [105]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT name AS league,
       AVG(home_goal + away_goal) AS avg_goals,
       RANK() OVER(ORDER BY AVG(home_goal + away_goal)) AS league_rank
FROM league l
LEFT JOIN filtered_match m 
ON l.id = m.country_id
WHERE season = '2011/2012'
GROUP BY league
ORDER BY league_rank;''', engine)

### **Flip OVER your results**

In the last exercise, the rank generated in your query was organized from smallest to largest. By adding `DESC` to your window function, you can create a rank sorted from largest to smallest.

```
SELECT 
    id,
    RANK() OVER(ORDER BY home_goal DESC) AS rank
FROM match;
```

* Complete the window function to rank each league from highest to lowest average goals scored.
* Order the main query by the rank you just created.

In [106]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT name AS league,
       AVG(home_goal + away_goal) AS avg_goals,
       RANK() OVER(ORDER BY AVG(home_goal + away_goal) DESC) AS league_rank
FROM league l
LEFT JOIN filtered_match m 
ON l.id = m.country_id
WHERE season = '2011/2012'
GROUP BY league
ORDER BY league_rank;''', engine)

# **OVER and PARTITION BY**

* Calculate separate values for different categories established in a partition
* Calculate different aggregate values within the same column, e.g.,

```
AVG(home_team_goal) OVER(PARTITION BY season)
```

## **Partition your data**

* *How many goals were scored in each match and how did that compare to the overall average?*




In [107]:
pd.read_sql_query('''SELECT date,
                            home_team_goal + away_team_goal AS goals,
                            AVG(home_team_goal + away_team_goal) OVER(PARTITION BY season) AS season_avg
                            FROM match
                            WHERE season IN ('2012/2013','2011/2012', '2013/2014');''', engine)

## **PARTITION BY multiple columns**



In [108]:
pd.read_sql_query('''SELECT DISTINCT name,
                            season,
                            --home_team_goal + away_team_goal AS goals,
                            ROUND(AVG(home_team_goal + away_team_goal) OVER(PARTITION BY season, name), 2) AS season_country_avg
                     FROM country c
                     LEFT JOIN match m ON c.id = m.country_id
                     WHERE season IN ('2011/2012', '2014/2015');''', engine)

### **PARTITION BY a column**

The `PARTITION BY` clause allows you to calculate separate "windows" based on columns you want to divide your results. For example, you can create a single column that calculates an overall average of goals scored for each season.

In this exercise, you will be creating a data set of games played by *Legia Warszawa* (Warsaw League), the top ranked team in Poland, and comparing their individual game performance to the overall average for that season.

Where do you see more outliers? Are they *Legia Warszawa*'s home or away games?

* Complete the two window functions that calculate the home and away goal averages. Partition the window functions by season to calculate separate averages for each season.
* Filter the query to only include matches played by Legia Warszawa, `id = 8673`.

In [109]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, season, home_goal, away_goal,
	   CASE WHEN hometeam_id = 8673 THEN 'home' ELSE 'away' END AS warsaw_location,
       AVG(home_goal) OVER(PARTITION BY season) AS season_homeavg,
       AVG(away_goal) OVER(PARTITION BY season) AS season_awayavg
FROM filtered_match
WHERE hometeam_id = 8673 OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC 
LIMIT 10;''', engine)

### **PARTITION BY multiple columns**

The `PARTITION BY` clause can be used to break out window averages by multiple data points (columns). You can even calculate the information you want to use to partition your data! For example, you can calculate *average goals scored by season and by country, or by the calendar year* (taken from the date column).

In this exercise, you will calculate the average number home and away goals scored *Legia Warszawa*, and their opponents, partitioned by the month in each season.

* `SELECT date, season, home_goal, away_goal, CASE WHEN hometeam_id = 8673 THEN 'home' ELSE 'away' END AS warsaw_location, ` from `filtered_match` CTE
         
* Construct two window functions partitioning the average of home and away goals by `season` and `month`.
* Filter the dataset by *Legia Warszawa*'s team ID (`8673`) so that the window calculation only includes matches involving them.

In [110]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, season, home_goal, away_goal,
	   CASE WHEN hometeam_id = 8673 THEN 'home' 
                                    ELSE 'away' END AS warsaw_location,
       AVG(home_goal) OVER(PARTITION BY season, STRFTIME('%m', date)) AS season_mo_home,
       AVG(away_goal) OVER(PARTITION BY season, STRFTIME('%m', date)) AS season_mo_away
FROM filtered_match
WHERE hometeam_id = 8673 OR awayteam_id = 8673
ORDER BY (home_goal + away_goal) DESC
LIMIT 10;''', engine)

# **Sliding Window Statement and Keywords**

`ROWS BETWEEN <start> AND <finish>` โดยที่ start หรือ finish มาจากคีย์เวิร์ดเหล่านี้

* `PRECEDING`: specify the number of rows before the current row
* `FOLLOWING`: specify the number of rows after the current row
* `UNBOUNDED PRECEDING`: specify every row since the beginnine
* `UNBOUNDED FOLOOWING`: specify every row until the end
* `CURRENT ROW`: Stop calculation at the current row

## **UNBOUNDED PRECEDING as running totals (cummulative)**

In [111]:
pd.read_sql_query('''
SELECT date,home_team_goal,away_team_goal, 
       SUM(home_team_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
FROM match
WHERE home_team_api_id = 8456 AND season = '2011/2012';''', engine)

## **PRECEDING**

* Calculate the score only the current and a previous match

In [112]:
pd.read_sql_query('''
SELECT date,home_team_goal,away_team_goal,
       SUM(home_team_goal) OVER(ORDER BY date ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS last2
FROM match
WHERE home_team_api_id = 8456
AND season = '2011/2012';''', engine)

### **Slide to the left**

Sliding windows allow you to create running calculations between any two points in a window using functions such as `PRECEDING`, `FOLLOWING`, and `CURRENT ROW`. You can calculate running counts, sums, averages, and other aggregate functions between any two points you specify in the data set.

In this exercise, you will expand on the examples discussed in the video, calculating the running total of goals scored by the *FC Utrecht* when they were the home team during the `2011/2012` season. Do they score more goals at the end of the season as the home or away team?

* `SELECT date, home_goal, away_goal FROM filtered_match` CTE
* Complete the window function by:
    * Assessing the running total of home goals scored by *FC Utrecht*.
    * Assessing the running average of home goals scored.
    * Ordering both the running average and running total by `date`.

In [113]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, home_goal, away_goal,
       SUM(home_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total,
       AVG(home_goal) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg
FROM filtered_match
WHERE hometeam_id = 9908 AND season = '2011/2012';''', engine)

### **Slide to the right**

Now let's see how *FC Utrecht* performs when they're the away team. You'll notice that the total for the season is at the bottom of the data set you queried. Depending on your results, this could be pretty long, and scrolling down is not very helpful.

In this exercise, you will slightly modify the query from the previous exercise by sorting the data set in ***reverse*** order and calculating a backward running total **from the** `CURRENT ROW`** to the end of the data set (earliest record).**

* Select the `date`, `home_goal`, and `away_goal` from `filtered_match` CTE
* Complete the window function by:
    * Assessing the running total of home goals scored by FC Utrecht.
    * Assessing the running average of home goals scored.
    * Ordering both the running average and running total by `date`, *descending*.

In [114]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT date, home_goal, away_goal,
       SUM(home_goal) OVER(ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_total,
       AVG(home_goal) OVER(ORDER BY date DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS running_avg
FROM filtered_match
WHERE awayteam_id = 9908 AND season = '2011/2012';''', engine)

### **Setting up the home team CTE**

In this course, we've covered ways in which you can use `CASE` statements, subqueries, common table expressions, and window functions in your queries to structure a data set that best meets your needs. For this exercise, you will be using all of these concepts to generate a list of matches in which Manchester United was defeated during the `2014/2015` English Premier League season.

Your first task is to create the first query that <ins>filters for matches where Manchester United played as the home team</ins>. This will become a common table expression in a later exercise.

* Create a `CASE` statement that identifies each match as a win, lose, or tie for Manchester United.
* Fill out the logical operators for each `WHEN` clause in the `CASE` statement (equals, greater than, less than).
* Join the tables on `hometeam_id` from `filtered_match` CTE, and `team_api_id` from `team`.
* Filter the query to only include games from the `2014/2015` season where Manchester United was the home team.

In [115]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, team_long_name,
       CASE WHEN home_goal > away_goal THEN 'MU home Win'
	        WHEN home_goal < away_goal THEN 'MU home Loss'
            ELSE 'Tie' END AS outcome
FROM filtered_match m
-- Left join team on the home team ID and team API id
LEFT JOIN team t ON m.hometeam_id = t.team_api_id
WHERE 
	-- Filter for 2014/2015 and Manchester United as the home team
	season = '2014/2015' AND team_long_name = 'Manchester United' ORDER BY m.id;''', engine)

### **Setting up the away team CTE**

Great job! Now that you have a query identifying the home team in a match, you will perform a similar set of steps to identify the away team. Just like the previous step, you will join the match and team tables. Each of these two queries will be declared as a Common Table Expression in the following step.

The primary difference in this query is that you will be joining the tables on `awayteam_id`, and reversing the match outcomes in the `CASE` statement.

When altering `CASE` statement logic in your own work, you can reverse either the logical condition (i.e., home_goal > away_goal) or the outcome in `THEN` -- just make sure you only reverse one of the two!

* `SELECT` match `id` and `team_long_name`
* Complete the `CASE` statement syntax.
* Fill out the logical operators identifying each match as a win, loss, or tie for *Manchester United*.
* Join the `filtered_match` CTE and `team` table on `awayteam_id`, and `team_api_id`.

In [116]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012'))
                               
SELECT m.id, team_long_name,
       CASE WHEN home_goal > away_goal THEN 'MU away Loss'
	        WHEN home_goal < away_goal THEN 'MU away Win'
            ELSE 'Tie' END AS outcome
FROM filtered_match m
LEFT JOIN team t ON m.awayteam_id = t.team_api_id
WHERE 
	-- Filter for 2014/2015 and Manchester United as the away team
	season = '2014/2015' AND team_long_name = 'Manchester United';''', engine)

### **Putting the CTEs together**

Now that you've created the two subqueries identifying the home and away team opponents, it's time to rearrange your query with the `home` and `away` subqueries as Common Table Expressions (CTEs). You'll notice that the main query includes the phrase, `SELECT DISTINCT`. Without identifying only `DISTINCT` matches, you will return a duplicate record for each game played.

Continue building the query to extract all matches played by ***Manchester United*** in the `2014/2015` season.

* Declare the `home` and `away` CTEs before your main query.
* Join your CTEs to the `filtered_match` CTE using a `LEFT JOIN`.
* Select the relevant data from the CTEs into the main query.
* Select the `date` from `filtered_match`, team names from the CTEs, and `home_goal` and `away_goal` from `filtered_match` in the main query.

In [117]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
                               
-- Set up the home team CTE
     home AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU home Win'
		                  WHEN home_goal < away_goal THEN 'MU home Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
-- Set up the away team CTE
     away AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal < away_goal THEN 'MU away Win'
		                  WHEN home_goal > away_goal THEN 'MU away Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.awayteam_id = t.team_api_id)
-- Select team names, the date and goals
SELECT DISTINCT date,
                home.team_long_name AS home_team,
                away.team_long_name AS away_team,
                home_goal,
                away_goal
-- Join the CTEs onto the match table
FROM filtered_match m
LEFT JOIN home USING(id)
LEFT JOIN away USING(id)
WHERE m.season = '2014/2015' AND (home.team_long_name = 'Manchester United' OR away.team_long_name = 'Manchester United');''', engine)

### **Add a window function**

Fantastic! You now have a result set that retrieves the match date, home team, away team, and the goals scored by each team. You have one final component of the question left -- *how badly did Manchester United lose in each match?*

In order to determine this, let's add a window function to the main query that ranks matches by the ***absolute value of the difference*** between `home_goal` and `away_goal`. This allows us to directly compare the difference in scores without having to consider whether Manchester United played as the home or away team!

* Set up the CTEs so that the home and away teams each have a name, ID, and score associated with them.
* Select the `date`, home team name, away team name, `home_goal`, and `away_goal`s scored in the main query.
* Rank the matches and order by the difference in scores in descending order.

In [118]:
pd.read_sql_query('''
WITH filtered_match AS (SELECT country_id,
                               season,
                               date,
                               id,
                               home_team_api_id AS hometeam_id,
                               away_team_api_id AS awayteam_id,
                               home_team_goal AS home_goal, 
                               away_team_goal AS away_goal 
                               FROM match 
                               WHERE season IN ('2013/2014', '2012/2013','2014/2015','2011/2012')),
    home AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU home Win'
		                  WHEN home_goal < away_goal THEN 'MU home Loss' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.hometeam_id = t.team_api_id),
    away  AS (SELECT m.id, team_long_name,
	                 CASE WHEN home_goal > away_goal THEN 'MU away Loss'
		                  WHEN home_goal < away_goal THEN 'MU away Win' 
  		                  ELSE 'Tie' END AS outcome
              FROM filtered_match m
              LEFT JOIN team t ON m.awayteam_id = t.team_api_id)

SELECT DISTINCT date,
                home.team_long_name AS home_team,
                away.team_long_name AS away_team,
                home_goal, away_goal,
                RANK() OVER(ORDER BY ABS(home_goal - away_goal) DESC) as match_rank
-- Join the CTEs onto the match table
FROM filtered_match m
LEFT JOIN home USING(id)
LEFT JOIN away USING(id)
WHERE season = '2014/2015'
AND ((home.team_long_name = 'Manchester United' AND home.outcome = 'MU home Loss') 
OR   (away.team_long_name = 'Manchester United' AND away.outcome = 'MU away Loss'));''', engine)